# Part II: Model Development

In this part, we develop three unique pipelines for predicting backorder. We use the smart sample from Part I to fit and evaluate these pipelines. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd
import joblib

from sklearn import preprocessing
from sklearn.model_selection import train_test_split


## Reload the smart sample here

In [12]:
# Reload your smart sampling from local file 
# ----------------------------------
X, y, train_undersamp = joblib.load('data/sample-data-v1.pkl')


## Normalize/standardize the data if required; otherwise ignore. You can perform this step inside the pipeline (if required). 

In [16]:
# Standardize data
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
''' # Combine X_scaled and y
train_under_stand = pd.DataFrame()
train_under_stand = X_scaled
train_under_stand.reset_index(drop = True)
train_under_stand['went_on_backorder'] = y '''

print("Mean of the dataset features")
print(scaler.mean_)
print("Variance of data")
print(scaler.scale_)
print("-" * 35)

print("# Scaled data:")
print(X_scaled)

print("# Mean of scaled data")
print(X_scaled.mean(axis = 0))
print("# Variance of scaled data")
print(X_scaled.std(axis = 0))


# Normalize data
X_normalized = preprocessing.normalize(X, axis = 0, norm = 'l2')  # axis = 0 means column-wise
''' # Combine X_normalized and y
train_under_norm = pd.DataFrame()
train_under_norm = X_normalized
train_under_norm['went_on_backorder'] = y '''

print('# Scaled & normalized values:')
print(X_normalized)

print("# All the line has unit norm")
print(np.linalg.norm(X_normalized, axis=0) )

Mean of the dataset features
[1.49783531e+02 7.01597165e+00 2.31098684e+01 1.49170901e+02
 3.75961981e+01 1.11286155e+02 3.39850410e+01 2.76166805e-03
 2.44987572e+00 7.41287858e-01 2.81487619e+00 1.85952315e-01
 5.98361410e-04 1.36242290e-01 9.74638682e-01 1.84111203e-04]
Variance of data
[2.05705652e+03 5.47785352e+00 4.83967322e+02 1.59776336e+03
 4.81613634e+02 1.40028856e+03 4.18233925e+02 5.24789599e-02
 4.14619611e+01 2.72113454e-01 6.30854082e+01 3.89068184e-01
 2.44541075e-02 3.43045666e-01 1.57219979e-01 1.35675092e-02]
-----------------------------------
# Scaled data:
[[-0.06989771 -0.91568196 -0.04775089 ... -0.39715497  0.16131104
  -0.01357001]
 [-0.06795318  0.90985061 -0.04775089 ...  2.51790883  0.16131104
  -0.01357001]
 [-0.06795318  0.17963758 -0.04775089 ... -0.39715497  0.16131104
  -0.01357001]
 ...
 [-0.07086997  0.17963758 -0.04775089 ... -0.39715497  0.16131104
  -0.01357001]
 [-0.06843931  0.90985061 -0.04775089 ... -0.39715497  0.16131104
  -0.01357001]
 [-

## Split the data into Train/Test

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


## Developing Pipeline

In this section, we design an operationalized machine learning pipeline, which includes:

* Anomaly detection
* Dimensionality Reduction
* Train a classification model


We are free to use any of the models that we learned in the past or we can use new models. Here is a pool of methods: 

### Pool of Anomaly Detection Methods (Discussed in M4)
1. IsolationForest
2. EllipticEnvelope
3. LocalOutlierFactor
4. OneClassSVM
5. SGDOneClassSVM

### Pool of Feature Selection Methods (Discussed in M3)

1. VarianceThreshold
1. SelectKBest with any scoring method (e.g, chi, f_classif, mutual_info_classif)
1. SelectKPercentile
3. SelectFpr, SelectFdr, or  SelectFwe
1. GenericUnivariateSelect
2. PCA
3. Factor Analysis
4. Variance Threshold
5. RFE
7. SelectFromModel


### Classification Methods (Discussed in M1-M2
1. Decision Tree
2. Random Forest
3. Logistic Regression
4. Naive Bayes
5. Linear SVC
6. SVC with kernels
7. KNeighborsClassifier
8. GradientBoostingClassifier
9. XGBClassifier
10. LGBM Classifier



It is difficult to fit an anomaly detection method in the sklearn pipeline without writing custom codes. For simplicity, we avoid fitting an anomaly detection method within a pipeline. So we can create the workflow in two steps. 
* Step I: fit an outlier with the training set
* Step II: define a pipeline using a feature selection and a classification method. Then cross-validate this pipeline using the training data without outliers. 
* Note: if your smart sample is somewhat imbalanced, you might want to change the scoring method in GridSearchCV (see the [doc](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).


Once we fit the pipeline with gridsearch, we identify the best model and give an unbiased evaluation using the test set that we created in Part II. For unbiased evaluation we report confusion matrix, precision, recall, f1-score, accuracy, and other measures if you like. 

**Optional: Those who are interested in writing custom codes for adding an outlier detection method into the sklearn pipeline, please follow this discussion [thread](https://stackoverflow.com/questions/52346725/can-i-add-outlier-detection-and-removal-to-scikit-learn-pipeline).**


**Note:** <span style='background:yellow'>We will be using Grid Search to find the optimal parameters of the pipelines.</span>

You can add more notebook cells or import any Python modules as needed.

In [18]:
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest

from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif

from sklearn.pipeline import Pipeline


### Your 1st pipeline 
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation
  
Add cells as needed. 

In [19]:
# Add anomaly detection code  (Question #E201)
# ----------------------------------
# Construct IsolationForest 
iso_forest = IsolationForest(contamination = 0.05).fit(X_train, y_train)

# Get labels from classifier and cull them
iso_outliers = iso_forest.predict(X_train) == -1
print(f"Number of outliers = {np.sum(iso_outliers)}")
X_iso = X_train[~iso_outliers]
y_iso = y_train[~iso_outliers]


Num of outliers = 869


In [20]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E202)
# ----------------------------------
#pca_components = 5    # Sets PCA components to 5

# Define pipeline
pipe = Pipeline([
    ('scale', MinMaxScaler()),                      # Scale the data
    ('PCA', PCA(n_components = pca_components)),    # reduce feature vector to size of 5
    ('grid', GridSearchCV(pipe, param_grid, n_jobs = 2, cv = 10))
])

# Define parameter grid
param_grid = {'PCA__n_components': [5, 20, 30, 35, 60], 
              #'Logistic__C': [0.001, 0.1, 1.0, 10, 100], 
              #'Logistic__max_iter': [1000]
              }

# Use Grid Search to train Pipeline
model_grid = GridSearchCV(pipe, param_grid, n_jobs = 2, cv = 10)

model_grid.fit(X_iso, y_iso)

SyntaxError: invalid syntax (<ipython-input-20-4269b6aafd9a>, line 10)

In [ ]:
# Given an unbiased evaluation  (Question #E203)
# ----------------------------------
# Show parameters of trained models and their rank
pd.set_option("max_colwidth", 80)
modelgrid_df = pd.DataFrame(model_grid.cv_results_)
modelgrid_df[['params','rank_test_score']]
#Show parameters of best model
best_params = model_grid.best_params_
print(best_params)

# Evaluate best model using test data
predicted_y = model_grid.predict(X_iso)

# Display confusion matrix
pd.DataFrame(confusion_matrix(y_test, predicted_y))

# Create classification report
print(classification_report(y_test, predicted_y))


#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 2nd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [ ]:
# Add anomaly detection code  (Question #E205)
# ----------------------------------


In [ ]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E206)
# ----------------------------------


In [ ]:
# Given an unbiased evaluation  (Question #E207)
# ----------------------------------

#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 3rd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [ ]:
# Add anomaly detection code  (Question #E209)
# ----------------------------------


In [ ]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E210)
# ----------------------------------


In [ ]:
# Given an unbiased evaluation  (Question #E211)
# ----------------------------------

#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## Compare these three pipelines and discuss your findings

## <span style="background: yellow;">Commit your code!</span> 

### Pickle the required pipeline/models for Part III.

You should have made a few commits so far of this project.  
**Definitely make a commit of the notebook now!**  
Comment should be: `Final Project, Checkpoint - Pipelines done`


# Save your notebook!
## Then `File > Close and Halt`